In [21]:
import os
import glob
import random
from os.path import exists
import pandas as pd
from sklearn.model_selection import train_test_split

In [22]:
REL_DATA_PATH = '../raw_gantt_chart_data_balanced/output/'
SOURCE_DIR = '../raw_gantt_chart_data_balanced/output/'
data_condition_dir = glob.glob(REL_DATA_PATH + '*')
data_condition_dir.sort()

RANDOM_STATE = random.seed(10)


DESTINATION_DIR = "../split_files/"
if not exists(DESTINATION_DIR):
    os.mkdir(DESTINATION_DIR)

In [23]:
wf_type_list   = ['1000genome','nowcast-clustering-16-casa_nowcast-wf','nowcast-clustering-8-casa_nowcast-wf',
            'wind-clustering-casa_wind_wf', 'wind-noclustering-casa_wind_wf']

all_subtypes = ['hdd_50', 'hdd_60', 'hdd_70', 'hdd_80', 'hdd_90', 'hdd_100','cpu_2', 'cpu_3',
                'loss_0.1', 'loss_0.5', 'loss_1', 'loss_3', 'loss_5','normal' ]

In [24]:
def split_data(wf_type_list, all_subtype, REL_DATA_PATH):
    
    train_fnames = []
    test_fnames  = []
    val_fnames   = []
    
    for wf_name in wf_type_list:
        for subtype in all_subtypes:
            type_path = REL_DATA_PATH + subtype + '/' + wf_name
            all_type_files = glob.glob(type_path + '*')
            train, rest = train_test_split(all_type_files, test_size=0.3, random_state=RANDOM_STATE, shuffle=True)
            train_fnames = train_fnames + train
            val, test = train_test_split(rest, test_size=0.5, random_state=RANDOM_STATE, shuffle=True)
            test_fnames = test_fnames + test
            val_fnames  = val_fnames + val
            
    return train_fnames, test_fnames, val_fnames

In [25]:
def save_splits(runs, DESTINATION_DIR, dataset):
    
    runs_df = pd.DataFrame(runs, columns=["path"])
    runs_df["label"] = [row.split('/')[3] for row in  runs_df["path"] ]
    runs_df["main_label"] = [row.split('_')[0] for row in  runs_df["label"] ]
    runs_df["filename"] = [row.split('/')[-1] for row in  runs_df["path"] ]
    runs_df.to_csv(DESTINATION_DIR + dataset + "_data.csv", index = False)
       
    return

In [26]:
train_fnames, test_fnames, val_fnames  = split_data(wf_type_list, all_subtypes, REL_DATA_PATH)

In [27]:
print(len(train_fnames))
print(len(test_fnames))
print(len(val_fnames)) 

3921
858
841


In [28]:
save_splits(train_fnames, DESTINATION_DIR, "train")
save_splits(val_fnames, DESTINATION_DIR, "validation")
save_splits(test_fnames, DESTINATION_DIR, "test")